# Notebook for Running Biomechanics Simulations with Reboot Motion Data

__[CoLab Notebook Link](https://githubtocolab.com/RebootMotion/reboot-toolkit/blob/main/examples_biomechanics/RebootMotionBiomechanicsSimulation.ipynb)__

Run the cells in order, making sure to enter AWS credentials in the cell when prompted

In [1]:
#@title Install Python Package

!pip install git+https://github.com/RebootMotion/reboot-toolkit.git@jacob/mujoco-batch-processing
!pip install git+https://github.com/RebootMotion/mlb-statsapi.git@v1.1.0#egg=mlb_statsapi > /dev/null
!echo "Done Installing Reboot Toolkit"

  Cloning https://github.com/RebootMotion/reboot-toolkit.git (to revision jacob/mujoco-batch-processing) to /tmp/pip-req-build-p811ksqo
  Running command git clone --filter=blob:none --quiet https://github.com/RebootMotion/reboot-toolkit.git /tmp/pip-req-build-p811ksqo
  Running command git checkout -b jacob/mujoco-batch-processing --track origin/jacob/mujoco-batch-processing
  Switched to a new branch 'jacob/mujoco-batch-processing'
  Branch 'jacob/mujoco-batch-processing' set up to track remote branch 'jacob/mujoco-batch-processing' from 'origin'.
  Resolved https://github.com/RebootMotion/reboot-toolkit.git to commit e7537f5aa443b79b0f944e6f643492f2bb9a2e69
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.0/395.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.

In [2]:
#@title Install MuJoCo Viewer

!pip install -q mjc_viewer
print("Done installing Mujoco Viewer")

Done installing Mujoco Viewer


In [3]:
#@title Import Python Libraries

import IPython
import matplotlib.pyplot as plt
import mujoco
import numpy as np
import os
import pandas as pd
from random import choice
import datetime

from IPython.display import display
from tqdm import tqdm

import reboot_toolkit as rtk

from reboot_toolkit import S3Metadata, MocapType, MovementType, Handedness, FileType, PlayerMetadata, setup_aws, decorate_primary_segment_df_with_stats_api

In [4]:
#@title AWS Credentials

# Upload your Organization's .env file to the local file system, per https://pypi.org/project/python-dotenv/
# OR input your credentials string generated by the Reboot Dashboard

boto3_session = setup_aws()

Org ID:
org-mlbbiomech

Current Boto3 Session:
Session(region_name='us-west-1')


In [5]:
#@title User Input - No code changes required below this section, just enter information in forms

# Update the below info to match your desired analysis information
# Common changes you might want to make:

# To analyze both Hawk-Eye HFR data from the Stats API,
# and also Hawk-Eye Action files (e.g. from the DSP),
#  set mocap_types=[MocapType.HAWKEYE_HFR, MocapType.HAWKEYE]

# To analyze baseball-hitting,
# set movement_type=MovementType.BASEBALL_HITTING

# To analyze right-handed players,
# set handedness=Handedness.RIGHT

# To analyze data from the momentum and energy files,
# set file_type=FileType.MOMENTUM_ENERGY

# See https://docs.rebootmotion.com/ for all available file types and the data in each
mocap_types = [MocapType.HAWKEYE_HFR, MocapType.HAWKEYE]
movement_type = MovementType.BASEBALL_PITCHING
handedness = Handedness.RIGHT
file_type = FileType.INVERSE_KINEMATICS

# Update the label to whatever you'd like to be displayed in the visuals
primary_segment_label = 'Primary Segment'

# Use this bool to add columns of data, like pitch_type and start_speed, from the stats API
add_stats_api = False  # True or False

if add_stats_api:
    print("Will add data from the Stats API like velo and pitch type")

else:
    print("Will NOT add data from the Stats API like velo and pitch type (set to True above if needed)")

Will NOT add data from the Stats API like velo and pitch type (set to True above if needed)


In [6]:
#@title The Function for Dynamically Simulating a Single Play with MuJoCo

from mjc_viewer import Serializer, Trajectory

def mujoco_sim(mj_model, mj_joint_names, positions_df, do_render=False):

    # set the suffix to be used for all resulting simulation columns
    col_suffix = "invdyn"

    # create the mujoco data element for the simulation
    mj_data = mujoco.MjData(mj_model)

    # set the simulation time step as the median time step in the play (all should be uniform)
    dt = positions_df['time'].diff().median()
    mj_model.opt.timestep = dt

    # reorder the IK data frame to make it easier to set the values in the simulation
    positions_df = rtk.reorder_joint_angle_df_like_model(
        mj_model, mj_data, positions_df.copy(), mj_joint_names
    )
    angle_cols = [
        col for col in list(positions_df) if not col.endswith('translation')
    ]
    positions_df[angle_cols] = positions_df[angle_cols].apply(np.radians)

    # compute the gradient of the positions
    # (the dynamic state of the model includes both positions and velocities)
    velocities_df = positions_df.copy().apply(np.gradient, raw=True) / dt

    sim_results = []

    if do_render:
        # create a Serializer and Trajectory instance
        serializer = Serializer(mj_model)
        trajectory = Trajectory(mj_data)

    else:
        trajectory = None
        serializer = None

    # Below is the most straightforward simulation loop with MuJoCo using "mj_step"
    # Here is documentation for the general simulation framework:
    # https://mujoco.readthedocs.io/en/latest/programming/simulation.html
    for i, row_pos in positions_df.iterrows():

        row_vel = velocities_df.iloc[i]

        mj_data.qpos = row_pos.to_numpy()

        mj_data.qvel = row_vel.to_numpy()

        mujoco.mj_step(mj_model, mj_data)

        mujoco.mj_inverse(mj_model, mj_data)

        if trajectory is not None:
            # store results in trajectory instance
            trajectory.step()

        # here you can save any simulation values you'd like...
        # see here for options: https://mujoco.readthedocs.io/en/stable/APIreference/APItypes.html#mjdata
        sim_results.append(mj_data.qfrc_inverse.copy())

    sim_results_df = pd.DataFrame(
        data=sim_results, columns=[f"{jn}_{col_suffix}" for jn in mj_joint_names]
    )

    if trajectory is not None:
        return sim_results_df, serializer, trajectory

    return sim_results_df

/usr/local/lib/python3.10/dist-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


In [7]:
#@title The Function for Running a Biomechanics Simulation for Each Play in a Data Segment

def simulate_segment_df(model_xml_str, segment_data_df):

    model = mujoco.MjModel.from_xml_string(model_xml_str)

    joint_names = rtk.get_model_info(model_xml_str, 'joint', return_names=True)

    sim_dfs = []

    print('Running a simulation for each play in the data segment...')
    for org_movement_id in tqdm(segment_data_df['org_movement_id'].unique()):

        ik_df = segment_data_df.loc[
            segment_data_df['org_movement_id'] == org_movement_id
        ].copy().reset_index(drop=True)

        current_sim_df = mujoco_sim(model, joint_names, ik_df)

        sim_dfs.append(current_sim_df)

    sim_df = pd.concat(sim_dfs, ignore_index=True)

    return pd.concat([segment_data_df, sim_df], axis=1)

In [8]:
#@title Set S3 File Info

s3_metadata = S3Metadata(
    org_id=os.environ['ORG_ID'],
    mocap_types=[MocapType.HAWKEYE_HFR, MocapType.HAWKEYE],
    movement_type=MovementType.BASEBALL_PITCHING,
    handedness=Handedness.RIGHT,
    file_type=FileType.INVERSE_KINEMATICS,
)

s3_df = rtk.download_s3_summary_df(s3_metadata)

Available data...

last_modified
['2023-04-14 21:47:24.888182' '2023-03-25 22:17:30.992199'
 '2023-04-14 21:51:34.391507' ... '2023-09-18 06:24:37.736017'
 '2023-09-18 13:18:15.711100' '2023-09-18 12:44:50.371546']

session_date
['2022-10-02T00:00:00.000000000' '2023-02-25T00:00:00.000000000'
 '2023-02-26T00:00:00.000000000' '2023-02-27T00:00:00.000000000'
 '2023-02-28T00:00:00.000000000' '2023-03-01T00:00:00.000000000'
 '2023-03-12T00:00:00.000000000' '2023-03-25T00:00:00.000000000'
 '2023-03-28T00:00:00.000000000' '2023-03-30T00:00:00.000000000'
 '2023-03-31T00:00:00.000000000' '2023-04-01T00:00:00.000000000'
 '2023-04-02T00:00:00.000000000' '2023-04-03T00:00:00.000000000'
 '2023-04-04T00:00:00.000000000' '2023-04-05T00:00:00.000000000'
 '2023-04-06T00:00:00.000000000' '2023-04-07T00:00:00.000000000'
 '2023-04-08T00:00:00.000000000' '2023-04-09T00:00:00.000000000'
 '2023-04-10T00:00:00.000000000' '2023-04-11T00:00:00.000000000'
 '2023-04-12T00:00:00.000000000' '2023-04-13T00:00:00.00

In [9]:
roster = pd.read_csv('demo-roster.csv')
roster

,Name,MLBAMID,weight
0,Zack Wheeler,554430,88.5
1,Spencer Strider,675911,88.5
2,Kevin Gausman,592332,93.0


In [10]:
if not os.path.exists('player_data'):
    os.mkdir('player_data')

In [11]:
roster_result_values = {}
for i in roster.index:
    player_meta_data = PlayerMetadata(
        org_player_ids=[roster.loc[i,'MLBAMID'].astype(str)],
        session_dates=None,
        session_nums=None,
        session_date_start=datetime.date(2023, 4, 1),
        session_date_end=datetime.date(2023, 9, 15),
        year=2023,
        org_movement_id=None, # set the play GUID for the skeleton animation; None defaults to the first play
        s3_metadata=s3_metadata,
    )

    primary_segment_summary_df = rtk.filter_s3_summary_df(player_meta_data, s3_df)

    primary_segment_data_df = rtk.load_sampled_games_to_df_from_s3_paths(
        primary_segment_summary_df['s3_path_delivery'].tolist(), add_ik_joints=True, add_elbow_var_val=True, game_proportion=0.25, movement_proportion=0.2
    )

    # Set the mass for the player in the primary segment (use kilograms)
    primary_player_mass = roster.loc[i,'weight']

    # Retrieve the scaled model (both in length and mass) from AWS
    primary_model_xml_str = rtk.scale_human_xml(primary_segment_data_df, primary_player_mass, boto3_session=boto3_session)

    # Simulate all the plays in the primary segment with the retrieved model
    primary_sim_data_df = simulate_segment_df(primary_model_xml_str, primary_segment_data_df)

    #primary_segment_dict = rtk.load_data_into_analysis_dict(
    #    primary_analysis_segment, primary_sim_data_df, segment_label=primary_segment_label
    #)

    # Filter dataframe
    filtered_df = primary_sim_data_df[(primary_sim_data_df['time_from_max_hand'] >= -0.2) & (primary_sim_data_df['time_from_max_hand'] <= 0.05)]

    # Group by and aggregate
    result = filtered_df.groupby(['session_num', 'org_movement_id']).agg({
        'right_shoulder_rot_invdyn': ['min', 'max'],
        'right_elbow_var_invdyn': ['min', 'max'],
        'right_elbow_invdyn': ['min', 'max']
    }).reset_index()

    # Flatten MultiIndex and rename columns
    result.columns = ['_'.join(col).strip() if col[1] else col[0] for col in result.columns.values]
    result.rename(columns={
        'right_shoulder_rot_invdyn_min': 'right_shoulder_rot_invdyn_min',
        'right_shoulder_rot_invdyn_max': 'right_shoulder_rot_invdyn_max',
        'right_elbow_var_invdyn_min': 'right_elbow_var_invdyn_min',
        'right_elbow_var_invdyn_max': 'right_elbow_var_invdyn_max',
        'right_elbow_invdyn_min': 'right_elbow_invdyn_min',
        'right_elbow_invdyn_max': 'right_elbow_invdyn_max'
    }, inplace=True)

    player_season_mean = result.median(numeric_only=True).round(1)
    player_season_std = result.mean(numeric_only=True).round(1)
    player_game_avg = result.drop(columns='org_movement_id').groupby('session_num').agg(['median', 'std']).round(1)

    player_game_avg.columns = ['_'.join(col).strip() if col[1] else col[0] for col in player_game_avg.columns.values]

    player_result_values = {}
    for key in player_season_mean.keys():
        player_result_values[f'{key}_median'] = player_season_mean[key]
        player_result_values[f'{key}_std'] = player_season_std[key]

    player_name = roster.loc[i,'Name']

    roster_result_values[player_name] = player_result_values

    player_game_avg.to_csv(f'player_data/{player_name}.csv')



Available data...

last_modified
['2023-04-02 05:32:01.316355' '2023-04-08 01:10:59.024669'
 '2023-04-13 10:17:04.237721' '2023-04-19 06:58:32.531033'
 '2023-04-24 16:57:06.120423' '2023-04-30 09:21:38.685217'
 '2023-05-06 17:23:15.092522' '2023-05-11 08:27:36.982032'
 '2023-05-17 12:08:07.567716' '2023-05-23 16:34:14.726111'
 '2023-05-28 09:29:11.843394' '2023-06-03 06:56:26.460111'
 '2023-06-09 06:08:40.166316' '2023-06-14 11:26:03.731842'
 '2023-06-19 05:45:12.799041' '2023-06-26 00:41:06.707423'
 '2023-07-02 08:22:39.894862' '2023-07-08 06:25:33.436655'
 '2023-07-17 11:07:13.265145' '2023-07-23 11:33:17.171536'
 '2023-07-29 09:01:18.679594' '2023-08-03 07:58:05.958000'
 '2023-08-09 05:23:35.858990' '2023-08-16 07:20:13.331345'
 '2023-08-27 14:10:03.854020' '2023-09-02 08:01:28.898060'
 '2023-09-07 11:44:27.888445' '2023-09-13 11:47:48.313918']

session_date
['2023-04-01T00:00:00.000000000' '2023-04-07T00:00:00.000000000'
 '2023-04-12T00:00:00.000000000' '2023-04-18T00:00:00.0000000

100%|██████████| 131/131 [00:24<00:00,  5.39it/s]


Available data...

last_modified
['2023-04-03 03:31:18.169160' '2023-04-07 11:45:18.955372'
 '2023-04-13 08:21:35.512567' '2023-04-20 18:44:03.506012'
 '2023-04-25 22:51:34.380941' '2023-05-02 05:22:42.969780'
 '2023-05-07 17:53:00.396098' '2023-05-13 12:45:44.907558'
 '2023-05-18 17:22:10.851693' '2023-05-24 09:34:38.564549'
 '2023-05-29 09:17:20.122219' '2023-06-04 13:00:44.182255'
 '2023-06-09 07:58:47.656479' '2023-06-15 22:44:47.318503'
 '2023-06-21 07:09:35.135086' '2023-06-27 05:21:54.078953'
 '2023-07-03 00:06:51.172420' '2023-07-09 06:57:10.690316'
 '2023-07-16 08:04:33.945650' '2023-07-21 00:18:03.984375'
 '2023-07-27 05:36:50.609042' '2023-08-02 07:48:48.419988'
 '2023-08-08 07:10:37.599406' '2023-08-13 11:11:02.443647'
 '2023-08-19 06:44:48.829815' '2023-08-26 12:22:20.201240'
 '2023-09-01 12:31:01.955078' '2023-09-07 06:21:37.258057'
 '2023-09-14 17:22:06.338375']

session_date
['2023-04-01T00:00:00.000000000' '2023-04-06T00:00:00.000000000'
 '2023-04-12T00:00:00.000000000

100%|██████████| 134/134 [00:25<00:00,  5.24it/s]


Available data...

last_modified
['2023-04-01 23:04:57.032360' '2023-04-12 04:51:29.421720'
 '2023-04-13 16:47:08.514164' '2023-04-18 06:41:11.940911'
 '2023-04-24 00:35:39.650315' '2023-04-30 00:18:57.991854'
 '2023-05-05 06:35:57.873255' '2023-05-11 07:36:58.204255'
 '2023-05-17 06:14:45.548369' '2023-05-22 01:37:43.012532'
 '2023-05-27 10:15:44.535023' '2023-06-02 11:00:01.690722'
 '2023-06-07 08:00:36.709213' '2023-06-12 01:22:56.596375'
 '2023-06-17 07:51:24.466807' '2023-06-21 23:40:49.418011'
 '2023-06-28 05:56:58.436001' '2023-07-03 00:52:55.332491'
 '2023-07-09 12:44:28.660543' '2023-07-23 10:39:11.619164'
 '2023-07-29 06:50:38.740219' '2023-08-04 01:51:35.330167'
 '2023-08-10 05:37:25.487436' '2023-08-17 08:33:57.838822'
 '2023-08-24 08:35:05.335988' '2023-08-29 06:39:18.845643'
 '2023-09-04 07:03:01.842694' '2023-09-10 06:42:30.333188'
 '2023-09-15 05:55:39.275669']

session_date
['2023-04-01T00:00:00.000000000' '2023-04-06T00:00:00.000000000'
 '2023-04-12T00:00:00.000000000

100%|██████████| 138/138 [00:26<00:00,  5.21it/s]


In [12]:
roster_result_values = pd.DataFrame.from_dict(roster_result_values,orient='index')

In [13]:
roster_result_values.to_csv('roster-results.csv')